# Menginterpretasikan Model

Anda dapat menggunakan Azure Machine Learning untuk menginterpretasikan model dengan menggunakan *explainer* yang mengkuantifikasi jumlah pengaruh yang disumbangkan setiap fitur pada label yang diprediksi. Ada banyak penjelasan umum, masing-masing cocok untuk berbagai jenis algoritma pemodelan; tetapi pendekatan dasar untuk menggunakannya adalah sama.

## Menginstal paket SDK

Selain versi terbaru paket **azureml-sdk** dan **azureml-widgets**, Anda memerlukan paket **azureml-explain-model** untuk menjalankan kode di notebook ini. Anda juga akan menggunakan perpustakaan Azure ML Interpretability (**azureml-interpret**). Anda dapat menggunakan ini untuk menafsirkan berbagai jenis model biasa, bahkan jika model tersebut belum dilatih dalam eksperimen Azure ML atau terdaftar di ruang kerja Azure ML.

Jalankan sel di bawah ini untuk memverifikasi bahwa paket-paket ini diinstal. 

In [ ]:
pip show azureml-explain-model azureml-interpret

## Menjelaskan model

Mari kita mulai dengan model yang dilatih di luar Azure Machine Learning - Jalankan sel di bawah ini untuk melatih model klasifikasi pohon keputusan.

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# Separate features and labels
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

Proses pelatihan menghasilkan beberapa metrik evaluasi model berdasarkan dataset validasi hold-back, sehingga Anda memiliki gagasan tentang seberapa akurat prediksinya; tetapi bagaimana fitur dalam data memengaruhi prediksi?

### Dapatkan penjelasan untuk model

Mari dapatkan penjelasan yang sesuai untuk model dari pustaka interpretabilitas Azure ML yang Anda instal sebelumnya. Ada banyak jenis eksplanatori. Dalam contoh ini Anda akan menggunakan *Tabular Explainer*, yang merupakan penjelasan "kotak hitam" yang dapat digunakan untuk menjelaskan berbagai jenis model dengan menjalankan [SHAP](https://github.com/slundberg/shap ) penjelas model.

In [ ]:
from interpret.ext.blackbox import TabularExplainer

# "features" and "classes" fields are optional
tab_explainer = TabularExplainer(model,
                             X_train, 
                             features=features, 
                             classes=labels)
print(tab_explainer, "ready!")

### Mendapatkan pentingnya fitur *global*

Hal pertama yang harus dilakukan adalah mencoba menjelaskan model dengan mengevaluasi *pentingnya fitur* secara keseluruhan - dengan kata lain, mengukur sejauh mana setiap fitur memengaruhi prediksi berdasarkan keseluruhan himpunan data pelatihan.

In [ ]:
# you can use the training data or the test data here
global_tab_explanation = tab_explainer.explain_global(X_train)

# Get the top features by importance
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

Kepentingan fitur diberi peringkat, dengan fitur yang paling penting dicantumkan pertama.

### Mendapatkan pentingnya fitur *lokal*

Jadi Anda memiliki pandangan keseluruhan, tetapi bagaimana dengan menjelaskan pengamatan individu? Mari buat penjelasan *lokal* untuk prediksi individu, dengan mengukur sejauh mana setiap fitur memengaruhi keputusan untuk memprediksi setiap kemungkinan nilai label. Dalam hal ini, ini adalah model biner, jadi ada dua kemungkinan label (non-diabetes dan diabetes); dan Anda dapat mengukur pengaruh setiap fitur untuk masing-masing nilai label ini untuk pengamatan individual dalam himpunan data. Anda hanya akan mengevaluasi dua kasus pertama dalam himpunan data pengujian.

In [ ]:
# Get the observations we want to explain (the first two)
X_explain = X_test[0:2]

# Get predictions
predictions = model.predict(X_explain)

# Get local explanations
local_tab_explanation = tab_explainer.explain_local(X_explain)

# Get feature names and importance for each possible label
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])

## Menambahkan kemampuan untuk menjelaskan eksperimen pelatihan model

Seperti yang Anda lihat, Anda dapat membuat penjelasan untuk model yang dilatih di luar Azure Machine Learning; tetapi saat Anda menggunakan eksperimen untuk melatih dan mendaftarkan model di ruang kerja Azure Machine Learning, Anda bisa membuat penjelasan model dan mencatatnya.

Jalankan kode di sel berikut untuk terhubung ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

### Melatih dan jelaskan model menggunakan eksperimen

Oke, mari buat eksperimen dan letakkan file yang dibutuhkan di folder lokal - dalam hal ini kita hanya akan menggunakan file CSV data diabetes yang sama untuk melatih model.

In [ ]:
import os, shutil
from azureml.core import Experiment

# Create a folder for the experiment files
experiment_folder = 'diabetes_train_and_explain'
os.makedirs(experiment_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(experiment_folder, "diabetes.csv"))

Sekarang kita akan membuat skrip pelatihan yang terlihat mirip dengan skrip pelatihan Azure ML lainnya kecuali yang menyertakan fitur berikut:

- Pustaka yang sama untuk menghasilkan penjelasan model yang kami gunakan sebelumnya diimpor dan digunakan untuk menghasilkan penjelasan global
- Pustaka **ExplanationClient** digunakan untuk mengunggah penjelasan ke hasil eksperimen

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Import Azure ML run library
from azureml.core.run import Run

# Import libraries for model explanation
from azureml.interpret import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# Separate features and labels
X, y = data[features].values, data['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# Get explanation
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# Get an Explanation Client and upload the explanation
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

# Complete the run
run.complete()

Eksperimen membutuhkan lingkungan Python untuk menjalankan skrip, jadi kami akan mendefinisikan spesifikasi Conda untuknya. Perhatikan bahwa pustaka **azureml-interpret** disertakan dalam lingkungan pelatihan sehingga skrip dapat membuat **TabularExplainer** dan menggunakan kelas **ExplainerClient**.

In [ ]:
%%writefile $experiment_folder/interpret_env.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- pip
- pip:
  - azureml-defaults
  - azureml-interpret

Sekarang Anda dapat menjalankan eksperimen.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails


# Create a Python environment for the experiment
explain_env = Environment.from_conda_specification("explain_env", experiment_folder + "/interpret_env.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                      script='diabetes_training.py',
                      environment=explain_env,
                      docker_runtime_config=DockerConfiguration(use_docker=True)) 

# submit the experiment
experiment_name = 'mslearn-diabetes-explain'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

## Mengambil nilai pentingnya fitur

Setelah percobaan berjalan selesai, Anda dapat menggunakan kelas **ExplanationClient** untuk mengambil pentingnya fitur dari penjelasan yang terdaftar untuk percobaan tersebut.

In [ ]:
from azureml.interpret import ExplanationClient

# Get the feature explanations
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## Melihat penjelasan model di studio Azure Machine Learning

Anda juga dapat mengeklik tautan **Lihat detail proses** di widget Run Details untuk melihat proses di studio Azure Machine Learning, dan melihat tab **Penjelasan**. Kemudian:

1. Pilih ID penjelasan untuk penjelasan tabular Anda.
2. Lihat bagan **Kepentingan fitur agregat**, yang menunjukkan pentingnya fitur global secara keseluruhan.
3. Lihat bagan **Kepentingan fitur individu**, yang menunjukkan setiap titik data dari data pengujian.
4. Pilih satu titik untuk melihat pentingnya fitur lokal untuk prediksi individual untuk titik data yang dipilih.
5. Gunakan tombol **Kohor Baru** untuk menentukan subset data dengan setelan berikut:
    - **Nama kohor himpunan data**: Di bawah 25 detik
    - **Pilih filter**: Himpunan data
        - Usia kurang dari 25 (Pastikan Anda menambahkan filter ini sebelum menyimpan kohor baru).
6. Buat kohor baru kedua bernama **25 dan lebih** dengan filter pada Usia lebih besar dari atau sama dengan 25.
6. Tinjau visualisasi **Kepentingan fitur agregat** dan bandingkan kepentingan fitur relatif untuk dua kohor yang telah Anda tetapkan. Kemampuan untuk membandingkan kohor memungkinkan untuk melihat bagaimana fitur mempengaruhi prediksi secara berbeda untuk beberapa himpunan bagian dari populasi data.



**Informasi Lebih Lanjut**: Untuk informasi selengkapnya tentang menggunakan penjelasan di Azure ML, lihat [dokumentasi](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability). 